In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
import tensorflow as tf
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
with open('/content/intents.json') as json_data:
  data =json.load(json_data)

JSONDecodeError: ignored

In [ ]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Harsh.',
    'Thanks for asking. I am designed by Harsh Roy.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from World's 

In [ ]:
words=[]
classes=[]
documents=[]
ignore=['?']
for intent in data['intents']:
    for pattern in intent['patterns']:
      w=nltk.word_tokenize(pattern)
      words.extend(w)
      documents.append(( w, intent['tag'] ))
      if intent['tag'] not in classes:
        classes.append(intent['tag'])

In [ ]:
stemmer=PorterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents),"documents")
print(len(classes),"classes",classes)
print(len(words),"unique stemmed words",words)

27 documents
8 classes ['about', 'anime', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'thanks']
52 unique stemmed words ["'s", 'about', 'account', 'address', 'ani', 'anim', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'favourit', 'give', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'media', 'name', 'out', 'reach', 'recommend', 'see', 'social', 'some', 'suggest', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']


In [ ]:
training=[]
output=[]
output_empty=[0]*len(classes)

for doc in documents:
  bag=[]
  pattern_words=doc[0]
  pattern_words=[stemmer.stem(w.lower()) for w in pattern_words ]

  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)


  output_row=list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag,output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-6-de0daf5f59b8>:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]),activation = 'softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(np.array(train_x),np.array(train_y),epochs=100, batch_size=8, verbose=1)
model.save('model.pkl')

Epoch 1/100
4/4 [==============================] - 1s 8ms/step - loss: 2.0081 - accuracy: 0.1481
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 1.9833 - accuracy: 0.2222
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 1.9632 - accuracy: 0.2222
Epoch 4/100
4/4 [==============================] - 0s 5ms/step - loss: 1.9436 - accuracy: 0.2222
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 1.9250 - accuracy: 0.2593
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 1.9054 - accuracy: 0.2593
Epoch 7/100
4/4 [==============================] - 0s 3ms/step - loss: 1.8873 - accuracy: 0.2963
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 1.8686 - accuracy: 0.2963
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.8508 - accuracy: 0.2963
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 1.8324 - accuracy: 0.2963
Epoch 11/100
4/4 [===========

In [ ]:
import pickle
pickle.dump({'words': words , 'classes':classes}, open("training_data","wb" ))

In [ ]:
from keras.models import load_model
model=load_model('model.pkl')

In [ ]:
def clean_up_sentence (sentence):
    sentence_words = nltk.word_tokenize (sentence)
    sentence_words = [stemmer.stem (word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
       for i,w in enumerate(words):
           if w== s:
              bag[i] = 1
    bag-np.array(bag)
    return(bag)

In [ ]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
   bag = bow(sentence, words)
   results = model.predict(np.array([bag]))
   results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
   results.sort(key=lambda x: x[1], reverse=True)
   return_list = []
   for r in results:
       return_list.append((classes[r[0]], [1]))
   return return_list

def response (sentence):
    results = classify(sentence)
    if results:
       while results:
            for i in data['intents']:
                  if i['tag'] ==results[0][0]:
                    return print(random.choice(i['responses']))
    results.pop(0)


In [ ]:
response('Who are you?')

1/1 [==============================] - 0s 90ms/step
Hi, I am Harsh. Nice to meet you. I made this chatbot for fun and practice.


In [ ]:
response('Where can we connect')

1/1 [==============================] - 0s 23ms/step
You can connect me on Linkedin or Github 
 Linkedin - https://www.linkedin.com/in/harsh-roy-1100b01bb/ 
 Github - https://github.com/harshroy7632


In [ ]:
response('Suggest me some anime')

1/1 [==============================] - 0s 23ms/step
Hunter x Hunter
